In [1]:
using LinearAlgebra, Distributions, Statistics
cd("../src/")
include("mvn.jl")
include("truncnorm.jl");

In [2]:
iters = 20
ns = 500:500:2500
ds = 4:4:20

ans_mc = zeros(length(ns), length(ds), iters)
ans_qmc = zeros(length(ns), length(ds), iters)
time_mc = zeros(length(ns), length(ds))
time_qmc = zeros(length(ns), length(ds))
real = zeros(length(ns), length(ds), iters);

In [3]:
for i in 1:length(ns)
    n = ns[i]
    for j in 1:length(ds)
        d = ds[j]
        Σ = Symmetric(Matrix(1.0LinearAlgebra.I(d)))
        μ = zeros(d) 
        for p in 1:iters
            a = fill(-Inf, d)
            b = zeros(d)
            time_mc[i, j] += @elapsed ans_mc[i, j, p] += cdf_trunnormal(a, b, μ, Σ, cnts = n)
            time_qmc[i, j] += @elapsed ans_qmc[i, j, p] += mvn(cholesky(Σ).L, a, b, ns = 10, N = Int(n/10), μ = μ)
            real[i, j, p] += (1/2^d)
        end
    end
end

In [4]:
mapslices(mean, abs.((ans_mc .- real) ./real), dims = [3])[:,:,1]

5×5 Array{Float64,2}:
 0.1224     0.568     1.6192   1.0  1.0
 0.0952     0.5064    1.9384   1.0  1.0
 0.0890667  0.387467  1.50187  1.0  1.0
 0.054      0.2648    1.024    1.0  1.0
 0.05088    0.31952   1.00112  1.0  1.0

In [5]:
time_mc / iters

5×5 Array{Float64,2}:
 0.0294369   0.00161216  0.00175785  0.00190384  0.00190533
 0.00459878  0.00414193  0.00276527  0.00277732  0.00341493
 0.00554504  0.0046416   0.00420812  0.00475908  0.0045468 
 0.00697469  0.00650891  0.00717504  0.0058015   0.0054994 
 0.0073133   0.00923492  0.00812341  0.00833119  0.0075597 

In [6]:
mapslices(mean, abs.((ans_qmc .- real) ./real), dims = [3])[:,:,1]

5×5 Array{Float64,2}:
 0.0  2.22045e-16  4.44089e-16  4.44089e-16  0.0
 0.0  2.22045e-16  4.44089e-16  2.22045e-16  0.0
 0.0  2.22045e-16  4.44089e-16  2.22045e-16  0.0
 0.0  2.22045e-16  2.22045e-16  2.22045e-16  0.0
 0.0  2.22045e-16  2.22045e-16  2.22045e-16  0.0

In [7]:
time_qmc / iters

5×5 Array{Float64,2}:
 0.0582784    0.000344842  0.000620749  0.000635939  0.00114078
 0.000346188  0.000883852  0.00127415   0.00168425   0.00204112
 0.000608265  0.00111924   0.00156333   0.0018929    0.00304108
 0.00110053   0.00122327   0.00132245   0.00251298   0.0035908 
 0.000874603  0.00217375   0.00331898   0.00375545   0.00559457